In [ ]:
####### LagLlama Indiv ######

from itertools import islice
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
from tqdm.autonotebook import tqdm
import torch
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.dataset.pandas import PandasDataset
from lag_llama.gluon.estimator import LagLlamaEstimator
from gluonts.dataset.common import ListDataset
import pandas as pd

exp = 'Exp25_2024'

import os
import csv

products = sorted([name for name in os.listdir('../uf/') if os.path.isdir(os.path.join('../uf/', name))])

def extract_estado(file_name):
    parts = file_name.split('_')
    estado = parts[1]
    return estado

def read_csv_files(folder_path):
    estados = []
    files = os.listdir(folder_path)
    for file_name in files:
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r', newline='') as csvfile:
                reader = csv.reader(csvfile)
                headers = next(reader)
                estado = extract_estado(file_name)
                estados.append(estado)
                estados.sort()
    return estados


def get_lag_llama_predictions(dataset, prediction_length, context_length=48, num_samples=300, device="cuda", batch_size=64, nonnegative_pred_samples=True): 
    ckpt = torch.load("lag-llama.ckpt", map_location=device)
    estimator_args = ckpt["hyper_parameters"]["model_kwargs"]
    estimator = LagLlamaEstimator(
        ckpt_path="lag-llama.ckpt",
        prediction_length=prediction_length,
        context_length=context_length,
        input_size=estimator_args["input_size"],
        n_layer=estimator_args["n_layer"],
        n_embd_per_head=estimator_args["n_embd_per_head"],
        n_head=estimator_args["n_head"],
        scaling=estimator_args["scaling"],
        time_feat=estimator_args["time_feat"],
        nonnegative_pred_samples=nonnegative_pred_samples,
        rope_scaling={
            "type": "linear",
            "factor": max(1.0, (context_length + prediction_length) / estimator_args["context_length"]),
        },
        batch_size=batch_size,
        num_parallel_samples=num_samples,
    )
    lightning_module = estimator.create_lightning_module()
    transformation = estimator.create_transformation()
    predictor = estimator.create_predictor(transformation, lightning_module)
    forecast_it, ts_it = make_evaluation_predictions(
        dataset=dataset,
        predictor=predictor,
        num_samples=num_samples
    )
    forecasts = list(tqdm(forecast_it, total=len(dataset), desc="Forecasting batches"))
    tss = list(tqdm(ts_it, total=len(dataset), desc="Ground truth"))
    return forecasts, tss

for product in products:
    folder_path = f'../../uf/{product}/'
    estados = read_csv_files(folder_path)
    for estado in estados:
        dataset_own = pd.read_csv(f"../uf/{product}/mensal_{estado}_{product}.csv", header=0, sep=";")
        target_values = dataset_own['m3']
        start_date = pd.to_datetime(dataset_own['timestamp'].iloc[0], format='%Y%m')
        prediction_length = 12
        train_dataset = ListDataset(
            [{"start": '1990-01', "target": target_values[:-prediction_length], "item_id": product}],
            freq="M"
        )
        test_dataset = ListDataset(
            [{"start": '1990-01', "target": target_values, "item_id": product}],
            freq="M"
        )
        context_length = prediction_length*3
        num_samples = 300
        device = "cuda"
        forecasts, tss = get_lag_llama_predictions(
            test_dataset,
            prediction_length=prediction_length,
            num_samples=num_samples,
            context_length=context_length,
            device=device
        )
        plt.figure(figsize=(20, 15))
        date_formater = mdates.DateFormatter('%b, %d')
        plt.rcParams.update({'font.size': 15})
        for idx, (forecast, ts) in islice(enumerate(zip(forecasts, tss)), 9):
            ax = plt.subplot(1, 1, idx+1)
            plt.plot(ts[-4 * prediction_length:].to_timestamp(), label="target", )
            forecast.plot( color='g')
            plt.xticks(rotation=60)
            ax.xaxis.set_major_formatter(date_formater)
            ax.set_title(forecast.item_id)
        plt.gcf().tight_layout()
        plt.legend()
        plt.savefig(f'./{exp}/forecast_plots7_{product}_{estado}.png') 
        ckpt = torch.load("lag-llama.ckpt", map_location=device)
        estimator_args = ckpt["hyper_parameters"]["model_kwargs"]
        estimator = LagLlamaEstimator(
                ckpt_path="lag-llama.ckpt",
                prediction_length=prediction_length,
                context_length=context_length,
                nonnegative_pred_samples=True,
                aug_prob=0.5,
                lr=5e-3,
                input_size=estimator_args["input_size"],
                n_layer=estimator_args["n_layer"],
                n_embd_per_head=estimator_args["n_embd_per_head"],
                n_head=estimator_args["n_head"],
                time_feat=estimator_args["time_feat"],
                batch_size=64,
                num_parallel_samples=num_samples,
                trainer_kwargs = {"max_epochs": 30,},
            )
        predictor = estimator.train(train_dataset, cache_data=True)
        forecast_it, ts_it = make_evaluation_predictions(
            dataset=test_dataset,
            predictor=predictor,
            num_samples=num_samples
        )
        forecasts = list(tqdm(forecast_it, total=len(test_dataset), desc="Forecasting batches"))
        tss = list(tqdm(ts_it, total=len(test_dataset), desc="Ground truth"))
        plt.figure(figsize=(20, 15))
        date_formater = mdates.DateFormatter('%b, %d')
        plt.rcParams.update({'font.size': 15})
        for idx, (forecast, ts) in islice(enumerate(zip(forecasts, tss)), 9):
            ax = plt.subplot(1, 1, idx+1)
            plt.plot(ts[-4 * prediction_length:].to_timestamp(), label="target", )
            forecast.plot( color='g')
            plt.xticks(rotation=60)
            ax.xaxis.set_major_formatter(date_formater)
            ax.set_title(forecast.item_id)
        plt.gcf().tight_layout()
        plt.legend()
        plt.savefig(f'./{exp}/forecast_plots8_{product}_{estado}.png')
        with open(f'./{exp}/LagLlama_{exp}.csv', 'a', newline='') as file:
            writer = csv.writer(file)
            temp = forecast.samples.mean(axis=0)
            values_list = temp.flatten().tolist()
            writer.writerow([product, estado, 'LagLlama', values_list])




/tmp/ipykernel_567778/2384952011.py:48: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("lag-llama.ckpt", map_location=device)
/home/user/miniconda3/envs/rap

Epoch 0: |          | 50/? [00:00<00:00, 57.36it/s, v_num=8166]

Epoch 0, global step 50: 'train_loss' reached 7.79764 (best 7.79764), saving model to '/home/user/Área de trabalho/LAGLLAMA/lag-llama2-FT/lightning_logs/version_8166/checkpoints/epoch=0-step=50.ckpt' as top 1


Epoch 1: |          | 50/? [00:00<00:00, 61.95it/s, v_num=8166]

Epoch 1, global step 100: 'train_loss' reached 6.90706 (best 6.90706), saving model to '/home/user/Área de trabalho/LAGLLAMA/lag-llama2-FT/lightning_logs/version_8166/checkpoints/epoch=1-step=100.ckpt' as top 1


...

Epoch 15: |          | 50/? [00:00<00:00, 57.86it/s, v_num=8174]

Epoch 15, global step 800: 'train_loss' reached 9.62420 (best 9.62420), saving model to '/home/user/Área de trabalho/LAGLLAMA/lag-llama2-FT/lightning_logs/version_8174/checkpoints/epoch=15-step=800.ckpt' as top 1


Epoch 16: |          | 50/? [00:00<00:00, 58.41it/s, v_num=8174]

Epoch 16, global step 850: 'train_loss' was not in top 1


Epoch 17: |          | 26/? [00:00<00:00, 59.55it/s, v_num=8174]